# Setup

In [ ]:
!pip install -qU openai pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 5.4 MB/s eta 0:00:00


This line of code installs two Python packages: `openai` and `pydantic`.

- The `!` symbol at the start indicates that this command is being run in a Jupyter notebook or another environment where shell commands can be executed directly.
- `pip install` is the command used to download and install packages from the Python Package Index (PyPI).
- `-q` flag stands for quiet mode, which suppresses the output of the installation process, making it less verbose.
- `-U` flag stands for upgrade, which ensures that the latest version of the package is installed, even if an older version already exists.
- `openai` is a Python library developed by OpenAI that allows interaction with their AI models and services.
- `pydantic` is a library used for building robust, scalable, and maintainable data validation and parsing. It provides runtime type checking and validation for Python data structures, making it useful in applications where data integrity is critical.

In [ ]:
# Standard library imports
import os
import time
from typing import Literal

# Third-party imports
from openai import OpenAI
from pydantic import BaseModel
from google.colab import userdata


This code segment imports necessary modules and classes from both the Python standard library and third-party libraries.

- **Standard library imports**:
  - `os`: This module provides a way to use operating system dependent functionality. It allows functions such as working with the file system, processes, and environment variables.
  - `time`: This module provides various time-related functions. It can be used for tasks like measuring the execution time of code segments, delaying execution, or handling dates and times.
  - `Literal` from `typing`: This is a type hint that indicates a variable can only take on specific literal values. It's often used with enums or when a variable should only have one of a few possible string or integer values.

- **Third-party imports**:
  - `OpenAI` from `openai`: This class is likely the main interface to OpenAI's services, allowing for interaction with their AI models and APIs.
  - `BaseModel` from `pydantic`: This is a base class provided by Pydantic for creating data models. It automatically generates validation logic based on type hints, making it easier to ensure data consistency and correctness.
  - `userdata` from `google.colab`: The import statement here seems incorrect or incomplete because `google.colab` does not typically have a module named `userdata`. Google Colab is an environment for running Jupyter notebooks in the cloud, and its API might be used differently. Typically, you would see imports like `from google.colab import drive` to mount Google Drive, or other modules for specific functionalities within Colab. The correct usage depends on what functionality is needed from Google Colab.



In [ ]:
class CFG:
    max_new_tokens = 100
    model= 'gpt-4o-mini'

This defines a class named `CFG`.

- `max_new_tokens = 100`: This is a class attribute that specifies the maximum number of new tokens to generate in a response. In the context of language models like those provided by OpenAI, a "token" can be a word, part of a word, or even a character, depending on how the model is configured. This setting controls how much text the model will produce when given a prompt.

- `model = 'gpt-4o-mini'`: This specifies the AI model to use for generating text. The name indicates it's a variant of the GPT-4 model, specifically a "mini" version, which implies it's a smaller or more efficient version of the full GPT-4 model.  

The `CFG` class seems to be used as a configuration or settings holder for an application that interacts with OpenAI's language generation capabilities. The attributes defined here can be accessed as class variables (e.g., `CFG.max_new_tokens`) and are used to configure how text is generated when interacting with the OpenAI model specified by the `model` attribute.

In [ ]:
client = OpenAI(api_key = userdata.get('openaivision'))

This line of code initializes an instance of the `OpenAI` class, which is used to interact with the OpenAI API. The `api_key` parameter is set to the value retrieved from the `userdata` dictionary using the key `'openaivision'`. This suggests that the API key for accessing OpenAI's services, specifically the vision model, is stored in the `userdata` dictionary under this key.

The `OpenAI` class is not a built-in Python class, so it must be imported from a library that provides an interface to the OpenAI API. The `api_key` parameter is required to authenticate requests made to the OpenAI API.

By setting up the client in this way, the code can use the instance to make calls to the OpenAI API, passing in the stored API key for authentication. This allows the code to access various OpenAI models and services, such as text or image generation, without having to manually handle API key management.

# Data

In [ ]:
reviews = [
   "The room was clean and the staff was friendly.",
   "The location was terrible and the service was slow.",
   "The food was amazing but the room was too small.",
]

# Basic eval

In [ ]:
SYSTEM_PROMPT = "You are a sentiment classifier assistant."
PROMPT_TEMPLATE = """
   Classify the sentiment of the following hotel review as positive, negative, or neutral:\n\n{review}
"""

These two lines define constants that will be used in a sentiment classification task.

The `SYSTEM_PROMPT` constant is a string that defines the role of the assistant. In this case, it specifies that the assistant is a "sentiment classifier assistant", indicating its purpose is to classify text based on sentiment.

The `PROMPT_TEMPLATE` constant is a multiline string that serves as a template for creating prompts to be used in sentiment classification tasks. The template includes a fixed part that provides context and instructions for the task, and a placeholder `{review}` where the actual hotel review to be classified will be inserted.

When a hotel review is provided, it can be formatted into this template by replacing `{review}` with the review text, resulting in a complete prompt that can be used as input to a sentiment classification model. For example:


In [ ]:
# normal eval
def classify_sentiment(review):
   response = client.beta.chat.completions.parse(
       model = CFG.model,
       messages=[
           {"role": "system", "content": SYSTEM_PROMPT},
           {"role": "user", "content": PROMPT_TEMPLATE.format(review=review)},
       ],
   )
   return response.choices[0].message

This function, `classify_sentiment`, takes a hotel review as input and returns the sentiment classification result. Here's how it works:

1. It sends a request to the OpenAI API using the `client.beta.chat.completions.parse` method.
2. The request includes two messages:
   - The first message has a "system" role and contains the content defined by the `SYSTEM_PROMPT` constant, which specifies that the assistant is a sentiment classifier.
   - The second message has a "user" role and contains the content created by formatting the `PROMPT_TEMPLATE` with the provided hotel review. This template includes instructions to classify the sentiment of the review as positive, negative, or neutral.
3. The request also specifies the model to use for parsing the messages, which is defined by the `CFG.model` variable.
4. The response from the OpenAI API contains a list of choices, where each choice represents a possible completion of the conversation.
5. The function returns the first message in the list of choices (`response.choices[0].message`), which should contain the sentiment classification result for the provided hotel review.

This approach effectively uses the OpenAI model to classify the sentiment of a given hotel review based on its content, with the `SYSTEM_PROMPT` and `PROMPT_TEMPLATE` providing context and guidance for the classification task.

In [ ]:
for review in reviews:
   sentiment = classify_sentiment(review)
   print(f"Review: {review}\nSentiment: {sentiment.content}\n")

Review: The room was clean and the staff was friendly.
Sentiment: The sentiment of the review is positive.

Review: The location was terrible and the service was slow.
Sentiment: The sentiment of the review is negative.

Review: The food was amazing but the room was too small.
Sentiment: The sentiment of the hotel review is mixed; however, if I have to classify it into one of the three categories, I would classify it as **neutral**. The reviewer expresses a positive sentiment about the food and a negative sentiment about the room size, balancing each other out.



This code iterates over a collection of hotel reviews (`reviews`) and classifies the sentiment of each one using the `classify_sentiment` function. For each review, it then prints out the original review text and its corresponding sentiment classification.

Here's a step-by-step breakdown:

1. The loop `for review in reviews:` iterates over each review in the `reviews` collection.
2. Within the loop, `sentiment = classify_sentiment(review)` calls the `classify_sentiment` function to classify the sentiment of the current review. The result is stored in the `sentiment` variable.
3. `print(f"Review: {review}\nSentiment: {sentiment.content}\n")` prints out two lines:
   - The first line displays the original review text, prefixed with "Review: ".
   - The second line displays the classified sentiment, prefixed with "Sentiment: ". The `.content` attribute is used to access the content of the `sentiment` message.
4. A newline character (`\n`) is added at the end of the print statement to separate each review's output.

This code effectively classifies and prints the sentiments of all reviews in the `reviews` collection, providing a clear and organized output that shows both the original review text and its corresponding sentiment classification.

# SO eval

In [ ]:
class SentimentResponse(BaseModel):
   sentiment: Literal["positive", "negative", "neutral"]

This code defines a class `SentimentResponse` using the `BaseModel` from a library such as Pydantic.

The `SentimentResponse` class has a single attribute `sentiment`, which is typed as a `Literal`. The `Literal` type means that the `sentiment` attribute can only take on one of the explicitly specified values, which in this case are:

- "positive"
- "negative"
- "neutral"

This implies that any instance of the `SentimentResponse` class will have a `sentiment` attribute with one of these three string values. The use of `Literal` provides strong typing and helps catch errors at runtime or during static type checking if an invalid value is assigned to the `sentiment` attribute.

For example, attempting to create an instance of `SentimentResponse` with an invalid sentiment value would result in an error:
```python
response = SentimentResponse(sentiment="unknown")  # This would raise a validation error
```
However, creating an instance with one of the allowed values would be valid:
```python
response = SentimentResponse(sentiment="positive")  # This is a valid sentiment value
```

In [ ]:
def classify_sentiment_with_structured_outputs(review):
   response = client.beta.chat.completions.parse(
       model= CFG.model,
       messages=[
           {"role": "system", "content": SYSTEM_PROMPT},
           {"role": "user", "content": PROMPT_TEMPLATE.format(review=review)},
       ],
       response_format=SentimentResponse
   )
   return response.choices[0].message


This function, `classify_sentiment_with_structured_outputs`, is similar to the previous `classify_sentiment` function. However, it includes an additional parameter `response_format` in the `client.beta.chat.completions.parse` method call.

The `response_format` parameter specifies that the response from the API should be parsed into an instance of the `SentimentResponse` class. This means that the API's response will be expected to conform to the structure defined by the `SentimentResponse` model, which includes a single attribute `sentiment` with one of the allowed values ("positive", "negative", or "neutral").

By specifying the `response_format`, this function provides stronger typing and validation for the response data. If the API's response does not match the expected structure (i.e., it does not contain a valid sentiment value), a validation error will be raised.

The rest of the function remains the same: it sends a request to the OpenAI API with the provided review, and returns the first message in the list of choices from the response. However, because of the `response_format` parameter, the returned message is expected to contain a structured sentiment classification result that conforms to the `SentimentResponse` model.

For example, the returned message might be an instance of `SentimentResponse` with a `sentiment` attribute set to one of the allowed values:
```python
response = SentimentResponse(sentiment="positive")
```
This allows for more robust and type-safe handling of the sentiment classification results.

In [ ]:
for review in reviews:
    sentiment = classify_sentiment_with_structured_outputs(review)
    print(f"Review: {review}\nSentiment: {sentiment.content}")
    print('-' * 10)

Review: The room was clean and the staff was friendly.
Sentiment: {"sentiment":"positive"}
----------
Review: The location was terrible and the service was slow.
Sentiment: {"sentiment":"negative"}
----------
Review: The food was amazing but the room was too small.
Sentiment: {"sentiment":"neutral"}
----------


This code iterates over a collection of hotel reviews (`reviews`) and classifies the sentiment of each one using the `classify_sentiment_with_structured_outputs` function. For each review, it then prints out the original review text and its corresponding sentiment classification.

Here's a step-by-step breakdown:

1. The loop `for review in reviews:` iterates over each review in the `reviews` collection.
2. Within the loop, `sentiment = classify_sentiment_with_structured_outputs(review)` calls the function to classify the sentiment of the current review and stores the result in the `sentiment` variable.
3. `print(f"Review: {review}\nSentiment: {sentiment.content}")` prints out two lines:
   - The first line displays the original review text, prefixed with "Review: ".
   - The second line displays the classified sentiment, prefixed with "Sentiment: ". The `.content` attribute is used to access the content of the `sentiment` message.
4. `print('-' * 10)` prints a horizontal line of 10 dashes, which serves as a separator between each review's output.
This approach takes advantage of the structured output provided by the `classify_sentiment_with_structured_outputs` function.